# Analyzing Used Car Listings on eBay Kleinanzeigen

We will work with a dataset of used cars from *eBay Kleinanzeigen*, a classifieds section of the German eBay website.

This dataset has originally been scraped and uploaded to Kaggle. The original dataset is not available on Kaggle anymore, but can be found [here](https://data.world/data-society/used-cars-data).

The version of the dataset we will work with is a sample of 50,000 data points that was prepared by [Dataquest](https://www.dataquest.io/) including simulating a less-cleaned version of the data.

The data dictionary provided with data is as follows:

- `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
- `name` - Name of the car.
- `seller` - Whether the seller is private or a dealer.
- `offerType` - The type of listing.
- `price` - The price on the ad to sell the car.
- `abtest` - Whether the listing is included in an A/B test.
- `vehicleType` - The vehicle Type.
- `yearOfRegistration` - The year in which which year the car was first registered.
- `gearbox` - The transmission type.
- `powerPS` - The power of the car in PS.
- `model` - The car model name.
- `kilometer` - How many kilometers the car has driven.
- `monthOfRegistration` - The month in which which year the car was first registered.
- `fuelType` - What type of fuel the car uses.
- `brand` - The brand of the car.
- `notRepairedDamage` - If the car has a damage which is not yet repaired.
- `dateCreated` - The date on which the eBay listing was created.
- `nrOfPictures` - The number of pictures in the ad.
- `postalCode` - The postal code for the location of the vehicle.
- `lastSeenOnline` - When the crawler saw this ad last online.

During this project, we will focus on cleaning the dataset and analyze the included car listings.

In [ ]:
import os
import sys
import logging
from pathlib import Path

import numpy as np

%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

## Load Data

In [ ]:
from carsales.datasets import load_autos

In [ ]:
autos = load_autos()

In [ ]:
autos.info()

In [ ]:
autos.head()

Most of the data is represented as string. There are some attributes (`vehicleType`, `gearbox`, `model`, `fuelType` and `notRepairedDamage`) which have missing values. We will need to deal with that later.

We will start by cleaning the column names to make the data easier to work with: we will use the Python's preferred [snakecase](https://en.wikipedia.org/wiki/Snake_case) style, and also rename some fields to make sense more understandable.

## Rename Columns

In [ ]:
autos.columns

In [ ]:
autos.columns = [
    'date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test', 'vehicle_type',
    'registration_year', 'gearbox', 'power_ps', 'model', 'odometer', 'registration_month',
    'fuel_type', 'brand', 'unrepaired_damage', 'ad_created', 'n_pictures',
    'postal_code', 'last_seen'
]

In [ ]:
autos.head()

We will now investigate the data in more detail.

First of all, we should remove text columns where all or almost all values are the same, as it often has no useful information for further analysis.

It is also important to check for numeric data stored as text: they can be cleaned and converted to an appropriate format.

In [ ]:
autos.describe(include='all')

In [ ]:
autos['odometer'].value_counts().sort_index()

The following columns have have mostly one value and can be safely dropped:
- `seller`: all but one ads are associated to a private seller,
- `offer_type`: all but one ads are associated to the same value `Angebot`,
- `n_pictures`: all data seem to have no pictures.

There are also a few columns that need more investigation:
- `price` has unrealistically low and high values: some ads display a price of 0 and 1 USD, and also more than 999,000 USD! Moreover, the data is represented as strings; it would be more convenient and appropriate to use float numbers,
- `registration_year` has also the same issue with unrealistic scenarios (1000, 1001, 1111, 1500, 1800, and some values are above 2800).
- `registration_month` has values ranging from 0 to 12. Since there are only 12 months in a year, we need to look more carefully at this issue, and decide which value to get rid of (0 or 12).

The `odometer` attribute needs to be converted to a numerical type (currently represented as strings).

Let's first drop attributes mentioned above that are not interesting for further analysis.

In [ ]:
autos.drop(columns=['seller', 'offer_type', 'n_pictures'], inplace=True)

We now investigate the `price` attribute and we will:
- remove any non-numeric character,
- convert the column to a numeric dtype,
- get rid of instances that have unrealistic values.

In [ ]:
from carsales.features import remove_chars

In [ ]:
autos['price'] = remove_chars(autos['price'], ['$', ','])

In [ ]:
autos['price'].head()

In [ ]:
autos['price'] = autos['price'].astype(float)

In [ ]:
autos['price'].head()

In [ ]:
autos['price'].value_counts().sort_index(ascending=False).head(20)

In [ ]:
autos['price'].value_counts().sort_index(ascending=True).head(20)

From the two cells above, we can make the following remarks:
- there is a huge gap between high prices: the next price after 350,000 USD is almost 1,000,000 USD, which seems unrealistic. We will drop any data point whose price is greater than 350,000 USD,
- around 1,500 cars have been given a 0 USD price; we will also drop these data points,
- some cars have a very low price such as 1 USD; it may sound weird when we first look at the data, but remember eBay is an auction site, thus there could be items with such a low opening bid.

In [ ]:
from carsales.features import remove_prices

In [ ]:
price_max = 350_000.0
price_min = 1.0

In [ ]:
autos['price'] = remove_prices(autos['price'], price_min, price_max)

In [ ]:
autos['price'].min(), autos['price'].max()

For the `odometer`, we will also remove any non-numeric character and convert the column to a numeric dtype. It seems all values are reasonable.

In [ ]:
autos['odometer'] = remove_chars(autos['odometer'], ['km', ','])

In [ ]:
autos.rename(columns={'odometer': 'odometer_km'}, inplace=True)

In [ ]:
autos.columns

In [ ]:
autos['odometer_km'] = autos['odometer_km'].astype(float)

In [ ]:
autos['odometer_km']

In [ ]:
autos['odometer_km'].min()

In [ ]:
autos['odometer_km'].max()

In [ ]:
autos['price'].describe()

In [ ]:
autos['odometer_km'].describe()

In [ ]:
autos['price'].value_counts(dropna=False)

We can see that there are about 1,500 ads without having a price. Let's see if we can find more information about these ads.

In [ ]:
autos[autos['price'].isnull()]

So far, we cannot conclude anything interesting concerning these rows. What we could do:
- directly remove these rows from the dataset. It seems reasonable since it is less than 3% of the entire dataset, and we can also notice that these rows have missing values in other attributes such as `gearbox`, `vehicle_type` and `model`. There are also some of them which have a 0 value for `power_ps`, which is not realistic,
- use some imputing technique to fill missing values. Even if it is possible to do that, considering the first remark it means we would also need to fill missing values in the other attributes, which might be time-consuming and not worth it.

## Explore Date Columns

Considering the dataset documentation, there are 5 columns that should represent date values:
- `date_crawled` - When this ad was first crawled. All field-values are taken from this date.
- `ad_created` - The date on which the eBay listing was created.
- `last_seen` - When the crawler saw this ad last online.
- `registration_year` - The year in which which year the car was first registered.
- `registration_month` - The month in which which year the car was first registered.

In [ ]:
date_columns = ['date_crawled', 'ad_created',  'last_seen', 'registration_year', 'registration_month']

In [ ]:
autos[date_columns].head()

In [ ]:
autos[date_columns].info()

We can see that both `registration_year` and `registration_month` are already represented as numeric values. However, the three other columns (`date_crawled`, `ad_created` and `last_seen`) are still in a string format: we need to convert them into a numerical representation so as to understand them quantitatively.

Considering the cells above, the format is `YYYY-MM-DD HH:MM:SS`. That is, the first 10 characters represent the day. We will use this information to calculate the statistical distribution of values in `date_crawled`, `ad_created` and `last_seen`.

### Statistical Distributions

In [ ]:
str_date_columns = ['date_crawled', 'ad_created', 'last_seen']

In [ ]:
from carsales.features import create_date_distrib

In [ ]:
create_date_distrib(autos['date_crawled'])

It seems that the statistical distribution of listings crawled is uniform (over days).

In [ ]:
create_date_distrib(autos['ad_created'])

However, there is a wide range of `ad_created` dates. We can notice that the majority of ads have been created between March 2016 and April 2016, which corresponds to the dates we observed for `date_crawled`. We can also see that some ads were almost 10 months old.

In [ ]:
create_date_distrib(autos['last_seen'])

The associated [dataset documentation](https://data.world/data-society/used-cars-data) mentions that:
> The fields lastSeen and dateCrawled could be used to estimate how long a car will be at least online before it is sold.

Thanks to this, it could be interesting to create a new feature which represents how long a car have been online before it has been sold.

We can see that more that 47% of the listings were last seen between the last three days (from 2016 April, 5th to 2016 April, 7th), with a spike on 2016 April, 6th with more than 22%. This could be explained by two things:
- there was a huge spike in sales; this is quite unrealistic since it is an auction site, so there is no special type of promotion,
- this in fact represents the crawling period endings, and are not related to car sales.

Let's now look at the `registration_year` in more detail. As this attribute is already represented with numeric type, we do not need any preprocessing.

### Deal with Incorrect Registration Years

In [ ]:
autos['registration_year'].describe()

As mentioned earlier, there are unrealistic values. We should remove the associated listings. We can also catch some outliers and unrealistic scenarios by looking simultaneously at the `last_seen` attribute: indeed, a car cannot be first registered after the listing was seen. We can also compare the registration year with `ad_created`: a car cannot be registered after the ad was created.

We can find [here](https://en.wikipedia.org/wiki/Car) that the first car was invented in 1886: it gives us an estimate to deal with unrealistic listings associated with long distant dates.

Before directly removing these listings, we should have a look at the proportion that they represent: if it is too high, we would loose a lot of data. If it is the case, we should try to find more custom logic (such as imputing) instead of getting rid of the outliers.

In [ ]:
min_acceptable_year = 1886  # Date the first car was invented

In [ ]:
last_seen_years = pd.to_datetime(autos['last_seen']).dt.year

In [ ]:
max_acceptable_year = last_seen_years.max()  # See explanations above

In [ ]:
outliers_year_mask = ~autos['registration_year'].between(min_acceptable_year, max_acceptable_year)

In [ ]:
outliers_year_mask.sum() / len(autos)

Less than 4% of the dataset corresponds to these outliers, so we can safely delete them without hurting the dataset size too much. Let's also look for other potential types of outliers (cf. explanations above).

In [ ]:
((last_seen_years < autos['registration_year']) & ~outliers_year_mask).sum()

There is no additional car whose registration year is greater that its associated `last_seen` value.

In [ ]:
(( pd.to_datetime(autos['ad_created']).dt.year < autos['registration_year']) & ~outliers_year_mask).sum()

And there is no additional car registered after the ad was created.

In [ ]:
from carsales.features import remove_registrations

In [ ]:
remove_registrations(autos, min_acceptable_year, max_acceptable_year, inplace=True)

In [ ]:
len(autos)

We can now look at the statiscal distribution of `registration_year`.

In [ ]:
autos['registration_year'].value_counts(normalize=True).sort_index()

We can clearly see that most of the listings corresponds to cars registered from the 90's to 2016. There is also a gap around the 80's.

## Explore Car Prices by Brand

It can be interesting to use data aggregation to better understand the `brand` attribute. In particular, we can focus our attention on the mean price for each brand. This will give us an idea about the most expensive brands on the eBay market.

Let's first check if there is any missing value regarding car brands.

In [ ]:
autos['brand'].isnull().any()

In [ ]:
autos['brand'].value_counts(normalize=True)

The top 3 brands are Volkswagen, BMW and Opel, with respectively around 21%, 11% and 10% of the listings. We can also notice that Volkswagen (top 1 brand) is highly above BMW and Opel.

We will only focus our attention on brands that have over 5% of total values: it is not very relevant to include all brands.

In [ ]:
brands = autos['brand'].value_counts(normalize=True)
brands = brands[brands >= 0.05].index

In [ ]:
brands

In [ ]:
autos['brand'].value_counts(normalize=True) >= 0.01

In [ ]:
from carsales.features import compute_brands_avg_feature

In [ ]:
brand_avg_prices = compute_brands_avg_feature(autos, brands, 'price')

In [ ]:
brand_avg_prices

We can clearly see that Audi, Mercedes-Benz and BMW are the most expensive brands. The next most expensive one is Volkswagen, with a gap of almost 3,000 USD with BMW. Finally, Ford and Opel seem to be less expensive. This would explain the popularity of Volkswagen (as seen above in the brands statistical distribution), corresponding to more than 20% of the listings on the website.

For these top 6 brands, let's use again aggregation to see if there is a link between average price and average mileage: we will compute the average mileage of the cars for each brand, and put the result together with the average price computed above.

## Explore Average Mileage by Brand

In [ ]:
brand_avg_mileages = compute_brands_avg_feature(autos, brands, 'odometer_km')

In [ ]:
brand_avg_mileages

In [ ]:
brand_avg_prices = pd.Series(brand_avg_prices)
brand_avg_mileages = pd.Series(brand_avg_mileages)

In [ ]:
prices_and_mileages = pd.DataFrame({
    'mean_price': brand_avg_prices,
    'mean_mileage': brand_avg_mileages
})

In [ ]:
prices_and_mileages

The range of car mileages does not have significant variations as it is the case in the prices by brand. Note that Pandas provides a convenient way to do aggregation and to compute the same results as above:

In [ ]:
# Only keep the listings associated with top brands
autos_top_brands = autos[autos['brand'].isin(brands)]

In [ ]:
autos_top_brands['brand'].value_counts()

In [ ]:
autos_top_brands.groupby('brand')[['price', 'odometer_km']].mean().round(decimals=2).set_index(prices_and_mileages.index)

## Translation: from German to English

Since this dataset comes from a German website, it is likely we might find some German words. We will start by looking at all relevant categorical columns and see if we can find German vocabulary. Here we use the word *relevant* to refer to categorical columns of interest for further data analysis: for example, we do not want to translate the cars' names or models.

In [ ]:
autos.info()

In [ ]:
autos.select_dtypes(include='object').columns

We can notice that even if we decided not to clean `model` attribute, there is still a specific value which could be translated to English (*andere*, which means *other*):

In [ ]:
autos['model'].unique()

Since we only want to replace this value in this specific column, we will do it separately, after having processed the other columns.

In [ ]:
categorical_columns = ['ab_test', 'vehicle_type', 'gearbox', 'fuel_type', 'unrepaired_damage']

In [ ]:
autos[categorical_columns]

It seems that all features in `categorical_columns` contain German words, except for `ab_test`. Let's double-check that.

In [ ]:
autos['ab_test'].value_counts()

Indeed, that's the case, so we will drop `ab_test` from our list of relevant features.

In [ ]:
categorical_columns.remove('ab_test')
categorical_columns

Let's now explore each categorical feature of interest in more detail.

In [ ]:
for column in categorical_columns:
    print(f"{column}: {autos[column].unique()}\n")

We will use a [translator](https://translate.google.com/?sl=de&tl=en&op=translate) from German to English to replace the words we caught above. In order to do that, we first create a mapping and we apply it on the dataset.

In [ ]:
from carsales.constants import mapping_ge_to_en

In [ ]:
autos[categorical_columns] = autos[categorical_columns].replace(mapping_ge_to_en)

In [ ]:
for column in categorical_columns:
    print(f"{column}: {autos[column].unique()}\n")

In [ ]:
autos['model'] = autos['model'].replace({'andere': 'other'})

In [ ]:
print(f"model: {autos['model'].unique()}")

## Convert Dates to Uniform Numeric Data

Let's again take a look at the date columns in the dataset.

In [ ]:
date_columns = ['date_crawled', 'ad_created', 'last_seen']
date_columns

In [ ]:
autos[date_columns].head()

Currently, the dates are represented using the format `YYYY-MM-DD`. We want to change it to `YYYYMMDD` which seems to be more commonly used in the field of data science. That is, we only need to remove the dashes. Instead of removing these dashes using string functionalities, we will directly use `pandas.dt` module, which is more appropriate as we are dealing with dates.

In [ ]:
date_format = "%Y%m%d %H:%M:%S"
for column in date_columns:
    autos[column] = pd.to_datetime(autos[column]).dt.strftime(date_format)

In [ ]:
autos[date_columns].head()

## Look for Particular Keywords

It may be interesting if we could find useful keywords in some already existing columns, in order to extract them as new columns. This would enrich our dataset and may be useful if we decide to apply a machine learning model in the future.

Let's look at the `name` attribute in more detail.

In [ ]:
autos['name'].head(20)                            

It seems that the first word of the name correspond to the associated brand car. Let's confirm or infirm our assumption.

In [ ]:
# Get all brands from the `brand` attribute
brands = autos['brand'].unique()

In [ ]:
brands

In [ ]:
# Extract the first word of each name.
# If we want to compare the names, we need to lowercase the strings.
brands_from_names = autos['name'].str.split("_").str[0].str.casefold()

In [ ]:
brands_from_names

In [ ]:
# Check if our assumption was right
autos.loc[~brands_from_names.isin(brands), ['name', 'brand']]

Since the dataframe above is not empty, our assumption was not right. More precisely, even if some words like "VW" with "volkswagen" are related, we can see that there are some other names which are not at all related with the `brand` attribute, like "Kleinwagen" with "renault".

In [ ]:
autos.head(20)

In [ ]:
autos.info()

In [ ]:
autos[autos['fuel_type'].isnull()]

We can see that the feature `unrepaired_damage` is the one which has the highest number of missing values across the whole dataset. This feature seems to be important, since a customer is more likely to buy cars that have no unrepaired damage. We can try to impute this attribute.

A first could be to compute some statistics for the cars that have an `unrepaired_damage` missing value, and see if it seems reasonable to use these statistics to fill the missing values.

In [ ]:
autos.loc[autos['unrepaired_damage'].isnull(), 'price'].describe()

We can clearly see there is a lot of variance. Looking more carefully at the percentiles, we can see that, among cars without value for `unrepaired_damage`, 75% have a price below 3,700 USD. That is, it seems that most of these cars are cheaper. This would mean that these cars are rather old, or they have some unrepaired damage but the owner of the listing decided not to mention that.

In [ ]:
len(autos.loc[autos['unrepaired_damage'] == 'no', 'price'])

Our technique is based on the price; indeed, a car is more likely to be cheap if it has unrepaired damage. What we could do is calculate the average price for each type of car (here, by type we mean repaired or unrepaired damage) for those we have the information (*i.e.* no missing value in `unrepaired_damage`). Since we already observed a variability in the car prices, we should also compute the variance for each type. Then, for each car having a `unrepaired_damage` missing value, we would look at its price (if available) and decide whether to attribute a `yes` or `no` label.

First, let's remove the listings whose we do not have the `price` value, as it is not useful to keep them in the dataset since the car price is an important feature.

In [ ]:
autos = autos[autos['price'].notnull()]

In [ ]:
autos.info()

Let's then extract the cars whose we have some information about the `unrepaired_damage` attribute. This will be useful to perform some aggregation.

In [ ]:
autos_damage_notnull = autos[autos['unrepaired_damage'].notnull()]

In [ ]:
autos_damage_notnull.head()

In [ ]:
# autos_damage_notnull.groupby('unrepaired_damage')[['price', 'odometer_km']].mean().round(decimals=2).set_index(prices_and_mileages.index)